In [2]:
%matplotlib inline
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### Predict which factors may play a role in someone exiting a bank. To do that we must look at all  the column and infer whether it will matter in classifying a new customer or not.  The information about a customer is entailed in columns 0 through 12 (RowNumber-EstimatedSalary), while the output (whether the customer exited or not) is stored in the 13th row (Exited). 

For as much as we care, neither the customer ID, nor the surname should matter in classification. 
Therefore, we will use columns 3 (CreditScore) inclusive through the 13th column (exclusive).


#### 1.Read the Dataset

In [3]:
dataset = pd.read_csv('bank.csv')

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null int64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


#### 2.Drop the columns which are unique for all users like IDs
#### 3.Distinguish the feature and target set 

In [21]:
# Dropping Surname too as it shouldnt matter to classification
X = dataset.drop(labels=['RowNumber', 'CustomerId', 'Surname','Exited'], axis=1) # Features
y = dataset['Exited'] # Target

In [22]:
X.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58


In [23]:
X = pd.get_dummies(X)

In [24]:
X.head(2)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,1,0


#### 4.Divide the data set into Train and test sets

In [25]:
# Splitting the dataset into the Training and Testing set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

In [28]:
X_train.shape

(8000, 13)

In [29]:
X_test.shape

(2000, 13)

#### 5.Normalize the train and test data

In [30]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [31]:
X_train

array([[ 0.16958176, -0.46460796,  0.00666099, ...,  1.74309049,
         1.09168714, -1.09168714],
       [-2.30455945,  0.30102557, -1.37744033, ..., -0.57369368,
        -0.91601335,  0.91601335],
       [-1.19119591, -0.94312892, -1.031415  , ..., -0.57369368,
         1.09168714, -1.09168714],
       ...,
       [ 0.9015152 , -0.36890377,  0.00666099, ..., -0.57369368,
        -0.91601335,  0.91601335],
       [-0.62420521, -0.08179119,  1.39076231, ...,  1.74309049,
         1.09168714, -1.09168714],
       [-0.28401079,  0.87525072, -1.37744033, ..., -0.57369368,
         1.09168714, -1.09168714]])

####  6.Initialize & build the model

In [32]:
import keras

Using TensorFlow backend.


In [33]:
from keras.models import Sequential
from keras.layers import Dense

In [62]:
# Initializing the ANN
classifier = Sequential()

In [63]:
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units)
classifier.add(Dense(6, activation = 'relu', input_dim = 13))

In [64]:
# Adding the second hidden layer
classifier.add(Dense(6, activation = 'relu')) 

In [65]:
# Adding the output layer
classifier.add(Dense(1, activation = 'sigmoid')) 

In [66]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [67]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
8000/8000 [==============================] - 2s 226us/step - loss: 0.5206 - acc: 0.7536
Epoch 2/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4404 - acc: 0.7972
Epoch 3/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4303 - acc: 0.8089
Epoch 4/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4226 - acc: 0.8151
Epoch 5/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4130 - acc: 0.8235
Epoch 6/100
8000/8000 [==============================] - 1s 104us/step - loss: 0.4032 - acc: 0.8259
Epoch 7/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.3961 - acc: 0.8270
Epoch 8/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.3906 - acc: 0.8316
Epoch 9/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.3884 - acc: 0.8341
Epoch 10/100
8000/8000 [============================

In [68]:
y_pred = classifier.predict(X_test)
print(y_pred)

[[0.3006759 ]
 [0.22624964]
 [0.11565939]
 ...
 [0.19176078]
 [0.22932306]
 [0.20662856]]


#### 8.Predict the results using 0.5 as a threshold 

In [69]:
y_pred = (y_pred > 0.5)
print(y_pred)

[[False]
 [False]
 [False]
 ...
 [False]
 [False]
 [False]]


#### 9.Print the Accuracy score and confusion matrix

In [71]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1508   87]
 [ 181  224]]


In [74]:
scores = classifier.evaluate(X_test, y_test)
scores[1]*100

2000/2000 [==============================] - 0s 18us/step


86.6